# Importamos las utilidades de `utils.py`

In [1]:
from _utils import *

# Using CoinMarketAPI


## Ahora que tenemos los `id` de las categorias que necesitamos (IA, Games, RWA, etc), obtenemos la data de las criptomonedas

In [3]:
# Leemos el archivo JSON de categorias
categories_coinmarket = read_json("../../metadata/coinmarket/categories.json")

In [5]:
cryptos_coinmarket = dict()
# Recorremos las categorias (IA, Gaming, RWA, etc)
for category in categories_coinmarket:
    # Realizamos la petición a la API para obtener las criptomonedas de la categoria
    cryptos = fetch_data(
        api=APIS.COINMARKET,
        uri="cryptocurrency/category",
        parameters={"id": category["id"], "limit": 350},
    )
    # Guardamos las criptomonedas en un diccionario usando la categoria como llave y la data como valor
    cryptos_coinmarket[category["title"]] = {"coins": cryptos["data"]["coins"]}

### Ahora obtener que exchanges tienen estas cryptomonedas

In [9]:
# Recorrer las categorias
for category in cryptos_coinmarket:
    # Obtenemos una lista de cryptos de la categoria
    cryptos: list = cryptos_coinmarket[category]["coins"]
    # Recorremos cada crypto
    for crypto in cryptos:
        # Usamos la API para obtener las exchanges asociadas a esa moneda
        exchanges = fetch_data(
            api=APIS.COINMARKET,
            uri="exchange/map",
            parameters={"crypto_id": crypto["id"]},
        )
        # Creamos una nueva llave/valor para la crypto actual con las exchanges
        crypto["exchanges"] = exchanges
        # Esperamos 3 segundos antes de hacer la siguiente petición (para evitar que la API nos bloquee)
        time.sleep(3)

In [ ]:
# Guardamos el diccionario en un archivo JSON
save_json(cryptos_coinmarket, f"../../data/coinmarket/cryptos.json")

### Ahora obtener que portfolios tienen esas cryptos

In [28]:
cryptos_names = set()  # Set para almacenar los nombres de las criptomonedas
# Recorrer las categorias
for category in cryptos_coinmarket:
    # Obtenemos una lista de cryptos de la categoria
    cryptos = cryptos_coinmarket[category]["coins"]
    # Recorremos cada crypto
    for crypto in cryptos:
        # Agregamos el nombre de la criptomoneda al set
        cryptos_names.add(crypto["name"])

In [5]:
# Leemos el archivo JSON de portfolios
portfolios = read_json("../../metadata/coinmarket/portfolios.json")

In [7]:
portfolios_coinmarket = dict() # Diccionario para guardar los portfolios
# Recorremos los portfolios
for portfolio in portfolios:
    # Realizamos la petición a la API para obtener las criptomonedas del portfolio
    cryptos = fetch_data(
        api=APIS.COINMARKET,
        uri="cryptocurrency/category",
        parameters={"id": portfolio["id"], "limit": 1000},
    )
    # Guardamos las criptomonedas en un diccionario usando el portfolio como llave y la data como valor
    portfolios_coinmarket[portfolio["title"]] = cryptos["data"]["coins"]
    # Esperamos 3 segundos antes de hacer la siguiente petición (para evitar que la API nos bloquee)
    time.sleep(3)
    

In [12]:
# Filtrar las cryptos de los portfolios usando el set de `cryptos_names`
for portfolio in portfolios_coinmarket:
    cryptos = portfolios_coinmarket[portfolio]
    cryptos = list(filter(lambda crypto: crypto["name"] in cryptos_names, cryptos))
    portfolios_coinmarket[portfolio] = cryptos

In [24]:
# Total de cryptos en las wallets
reduce(lambda x, y: x + len(portfolios_coinmarket[y]), portfolios_coinmarket, 0)

292

In [27]:
# Guardar los datos en archivos JSON
save_json(portfolios_coinmarket, f"../../data/coinmarket/portfolios.json")